In [1]:
import multinetx as mx
import numpy as np
import random
from ling_model import create_MultilayerGraph

思路流程：
首先分别计算出上层和下层每个节点的度，从大到小，并且给到一个numpy数组中
注意：这里的度不包括层间的，所以需要减一
然后，按照式子进行计算

In [2]:
N = 16
mg = create_MultilayerGraph(N=N, p_g1=0.3, p_g2=0.4, seed_g1=28, seed_g2=25, intra_layer_edges_weight=2, inter_layer_edges_weight=3)

In [3]:
def get_degree_by_node(Graph, node, N, layer=0):
    return Graph.degree(N*layer + node) - 1

In [4]:
def get_degrees_by_layer(Graph, N, layer):
    degrees = []
    for i in range(N):
        degrees.append(get_degree_by_node(Graph, i, N, layer))
    return degrees

In [5]:
get_degrees_by_layer(mg, N, 1)

[6, 6, 6, 4, 7, 5, 5, 6, 7, 5, 9, 7, 7, 6, 5, 7]

分子的第一个参数

In [6]:
num1 = np.array(get_degrees_by_layer(mg, N, 0))
num2 = np.array(get_degrees_by_layer(mg, N, 1))

In [7]:
num1

array([10,  6,  6,  5,  3,  8,  1,  6,  5,  4,  4,  5,  5,  3,  3,  2])

In [8]:
num2

array([6, 6, 6, 4, 7, 5, 5, 6, 7, 5, 9, 7, 7, 6, 5, 7])

In [9]:
num1*num2

array([60, 36, 36, 20, 21, 40,  5, 36, 35, 20, 36, 35, 35, 18, 15, 14])

In [10]:
np.sum(num1*num2)/N

28.875

In [11]:
def get_avg_degree_product(array1, array2, N):
    return np.sum(array1*array2)/N

In [12]:
get_avg_degree_product(num1, num2, N)

28.875

分子的第二个参数

In [13]:
np.sum(num1)/N

4.75

In [14]:
np.sum(num2)/N

6.125

In [15]:
np.sum(num1)/N * np.sum(num2)/N

29.09375

In [16]:
def get_avg_degree(array, N):
    return np.sum(array)/N

分子的结果

In [17]:
molecular = get_avg_degree_product(num1, num2, N) - get_avg_degree(num1, N) * get_avg_degree(num2, N)

分母的参数

a层的

In [18]:
a1 = get_avg_degree_product(num1, num1, N)

In [19]:
a2 = np.square(get_avg_degree(num1, N))

In [20]:
a_num = np.sqrt(a1 - a2)

In [21]:
a_num

2.165063509461097

In [22]:
a1

27.25

In [23]:
a2

22.5625

In [24]:
def get_deominator_one(array, N):
    a1 = get_avg_degree_product(array, array, N)
    a2 = np.square(get_avg_degree(array, N))
    return np.sqrt(a1 - a2)

In [25]:
get_deominator_one(num1, N)

2.165063509461097

b层的

In [26]:
get_deominator_one(num2, N)

1.165922381636102

分母的结果

In [27]:
denominator = get_deominator_one(num1, N) * get_deominator_one(num2, N)

### 式子的最后结果

In [28]:
molecular / denominator

-0.08665782448262419

# 最后计算Pearson的函数

In [30]:
def get_degree_by_node(Graph, node, N, layer=0):
    return Graph.degree(N*layer + node) - 1
def get_degrees_by_layer(Graph, N, layer):
    degrees = []
    for i in range(N):
        degrees.append(get_degree_by_node(Graph, i, N, layer))
    return degrees
def get_avg_degree_product(array1, array2, N):
    return np.sum(array1*array2)/N
def get_avg_degree(array, N):
    return np.sum(array)/N
def get_deominator_one(array, N):
    a1 = get_avg_degree_product(array, array, N)
    a2 = np.square(get_avg_degree(array, N))
    return np.sqrt(a1 - a2)
def pearson_correlation_coefficient(Graph, N):
    # 首先判断网络是否是符合要求的网络
    assert mx.is_connected(Graph) == True, \
        "The network must be reachable.提示：遇到这种问题，一般是生成的随机化网络不满足条件，解决办法是重新运行或者在创建双层网络时增加边出现的概率。"
    assert Graph.is_directed() == False, \
        "It is only allowed for undirected complex network."
    assert len(Graph) > 3, \
        "Graph has less than three nodes."
    # 得到上下层的度numpy数组
    array1 = np.array(get_degrees_by_layer(Graph, N, 0))
    array2 = np.array(get_degrees_by_layer(Graph, N, 1))
    # 计算分子
    molecular = get_avg_degree_product(array1, array2, N) - get_avg_degree(array1, N) * get_avg_degree(array2, N)
    # 计算分母
    denominator = get_deominator_one(array1, N) * get_deominator_one(array2, N)
    return molecular / denominator

In [31]:
pearson_correlation_coefficient(mg, N)

-0.08665782448262419

In [32]:
type(np.array([]))

numpy.ndarray